Useful links:


- [AZ Py SDK Hyperparameter Tuning](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#sample-notebook)


- [AZ Py SDK Hyperparameter Notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/ml-frameworks/scikit-learn/train-hyperparameter-tune-deploy-with-sklearn/train-hyperparameter-tune-deploy-with-sklearn.ipynb)


In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
experiment_name = "udacity-project"

exp = Experiment(workspace=ws, name= experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


Workspace name: quick-starts-ws-159783
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-159783


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###



cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)


cpu_cluster.wait_for_completion(show_output=True)

    
    


Found existing cluster, use it.


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os



# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.01,0.5),
    "--max_iter": choice(100,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,
                      slack_factor=0.3,
                      delay_evaluation=3)


# * create a training folder
project_folder = "./training"
if "training" not in os.listdir():
    os.mkdir(project_folder)



'./training/train.py'

In [8]:
# Create a SKLearn estimator for use with train.py


est = SKLearn(source_directory='./',
              compute_target=cpu_cluster,
              entry_script='train.py',
              script_params={'--C':1,'--max_iter':100})



# Policy not working
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     policy=policy,
                                     hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()


hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c18f9708-2277-409c-8457-631f65f2a0f6
Web View: https://ml.azure.com/runs/HD_c18f9708-2277-409c-8457-631f65f2a0f6?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-159783/workspaces/quick-starts-ws-159783&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-10-01T21:25:21.071195][API][INFO]Experiment created<END>\n""<START>[2021-10-01T21:25:22.011460][GENERATOR][INFO]Trying to sample '1' jobs from the hyperparameter space<END>\n""<START>[2021-10-01T21:25:22.196849][GENERATOR][INFO]Successfully sampled '1' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_c18f9708-2277-409c-8457-631f65f2a0f6
Web View: https://ml.azure.com/runs/HD_c18f9708-2277-409c-8457-631f65f2a0f6?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-159783/workspaces/quick-starts-ws-159783&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



In [27]:
# Get your best run and save the model from that run.



best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.id)

best_run_metric = best_run.get_metrics()
print(best_run_metric['Accuracy'])



model = best_run.register_model(model_name='LogReg_HyperTuned', model_path="logs/azureml") 

/mnt/batch/tasks/shared/LS_root/mounts/clusters/udacity-compute/code/Users/odl_user_159783/udacity/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files-master
['--C', '94.64498558596091', '--max_iter', '100']


In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

web_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_aml = TabularDatasetFactory.from_delimited_files(path = web_path)

In [21]:
from train import clean_data

# Use the clean_data function to clean your data.
x,y = clean_data(data_aml)

import pandas as pd

dataset = pd.concat([x,y],axis=1)

In [ ]:
from azureml.core import Dataset

dataset.to_csv('data.csv',index = False)

datastore = ws.get_default_datastore()

datastore.upload_files(files = ['./data.csv'])

data = Dataset.Tabular.from_delimited_files([(datastore,'data.csv')])

In [29]:
from azureml.train.automl import AutoMLConfig
import logging

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_settings = {
    "n_cross_validations": 3,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4, # This is a limit for testing purpose, please increase it as per cluster size
    "experiment_timeout_minutes": 30, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ablity to find the best model possible
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             max_cores_per_iteration=-1,
                             compute_target = cpu_cluster,
                             training_data = data,
                             label_column_name = "y",
                             **automl_settings
                            )

In [30]:
# Submit your automl run

exp_aml = Experiment(ws, "aml")

aml_run = exp_aml.submit(automl_config, show_output = True)

from azureml.widgets import RunDetails
RunDetails(aml_run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_4dd54b05-e7a4-44cc-a831-54ec163e2983,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_4dd54b05-e7a4-44cc-a831-54ec163e2983',
 'target': 'cpucluster',
 'status': 'Completed',
 'startTimeUtc': '2021-10-01T19:48:14.343785Z',
 'endTimeUtc': '2021-10-01T20:11:57.437502Z',
 'services': {},
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpucluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-159783","workspace_name":"quick-starts-ws-159783","region":"southcentralus","compute_target":"cpucluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metri

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, aml_model = aml_run.get_output()

print(best_run)


best_run.register_model(model_name='AML', model_path="logs/azureml")


In [ ]:
cpu_cluster.delete()